In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from wordcloud import WordCloud

import nltk
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence #unique id

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Dropout, Embedding
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('/content/cyberbullying_tweets.csv')
df

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying
...,...,...
47687,"Black ppl aren't expected to do anything, depe...",ethnicity
47688,Turner did not withhold his disappointment. Tu...,ethnicity
47689,I swear to God. This dumb nigger bitch. I have...,ethnicity
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,ethnicity


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47692 entries, 0 to 47691
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tweet_text          47692 non-null  object
 1   cyberbullying_type  47692 non-null  object
dtypes: object(2)
memory usage: 745.3+ KB


In [4]:
df['cyberbullying_type'].value_counts()

religion               7998
age                    7992
gender                 7973
ethnicity              7961
not_cyberbullying      7945
other_cyberbullying    7823
Name: cyberbullying_type, dtype: int64

In [5]:
df['cyberbullying_type'] = df['cyberbullying_type'].replace({'religion': 0, 'gender': 0,'not_cyberbullying': 1,})

values = ['age', 'ethnicity', 'other_cyberbullying']
df = df[~df['cyberbullying_type'].isin(values)]

print(df['cyberbullying_type'].value_counts())


0    15971
1     7945
Name: cyberbullying_type, dtype: int64


In [6]:
df.shape

(23916, 2)

In [7]:
df['tweet_text'][0]

'In other words #katandandre, your food was crapilicious! #mkr'

In [8]:
def cleantext(text):
  tokens = word_tokenize(text.lower())
  ftoken = [t for t in tokens if(t.isalpha())]
  stop = stopwords.words("english")
  ctoken = [t for t in ftoken if(t not in stop)]
  lemma = WordNetLemmatizer()
  ltoken = [lemma.lemmatize(t) for t in ctoken]
  return " ".join(ltoken)

In [9]:
df["clean_text"]=df["tweet_text"].apply(cleantext)

In [10]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['cyberbullying_type'] = label_encoder.fit_transform(df['cyberbullying_type'])
df['cyberbullying_type']

0        1
1        1
2        1
3        1
4        1
        ..
23911    0
23912    0
23913    0
23914    0
23915    0
Name: cyberbullying_type, Length: 23916, dtype: int64

In [11]:
df.head()

,tweet_text,cyberbullying_type,clean_text
0,"In other words #katandandre, your food was cra...",1,word katandandre food crapilicious mkr
1,Why is #aussietv so white? #MKR #theblock #ImA...,1,aussietv white mkr theblock imacelebrityau tod...
2,@XochitlSuckkks a classy whore? Or more red ve...,1,xochitlsuckkks classy whore red velvet cupcake
3,"@Jason_Gio meh. :P thanks for the heads up, b...",1,meh p thanks head concerned another angry dude...
4,@RudhoeEnglish This is an ISIS account pretend...,1,rudhoeenglish isi account pretending kurdish a...


In [12]:
x = df['clean_text']
y = df['cyberbullying_type']

In [13]:
xtrain, xtest, ytrain, ytest = train_test_split(x,y, test_size=0.3, random_state=1)

In [14]:
ytest

13014    0
8705     0
4353     1
20707    0
6508     1
        ..
19123    0
1123     1
10815    0
19630    0
12714    0
Name: cyberbullying_type, Length: 7175, dtype: int64

In [15]:
sentlen = []

for sent in df["clean_text"]:
  sentlen.append(len(word_tokenize(sent)))     #sentlen.append(len(i.split(sent)))

df["SentLen"] = sentlen
df.head()

,tweet_text,cyberbullying_type,clean_text,SentLen
0,"In other words #katandandre, your food was cra...",1,word katandandre food crapilicious mkr,5
1,Why is #aussietv so white? #MKR #theblock #ImA...,1,aussietv white mkr theblock imacelebrityau tod...,10
2,@XochitlSuckkks a classy whore? Or more red ve...,1,xochitlsuckkks classy whore red velvet cupcake,6
3,"@Jason_Gio meh. :P thanks for the heads up, b...",1,meh p thanks head concerned another angry dude...,9
4,@RudhoeEnglish This is an ISIS account pretend...,1,rudhoeenglish isi account pretending kurdish a...,9


In [16]:
max(sentlen)

146

In [17]:
max_len = np.quantile(sentlen, 0.95)
max_len

27.0

In [18]:
tok = Tokenizer(char_level=False, split=" ")
#char_level	if True, every character will be treated as a token.

tok.fit_on_texts(xtrain)
tok.index_word

{1: 'joke',
 2: 'muslim',
 3: 'rape',
 4: 'gay',
 5: 'idiot',
 6: 'u',
 7: 'like',
 8: 'http',
 9: 'woman',
 10: 'people',
 11: 'mkr',
 12: 'rt',
 13: 'christian',
 14: 'amp',
 15: 'right',
 16: 'terrorist',
 17: 'make',
 18: 'one',
 19: 'call',
 20: 'islamic',
 21: 'get',
 22: 'know',
 23: 'islam',
 24: 'think',
 25: 'support',
 26: 'terrorism',
 27: 'bitch',
 28: 'say',
 29: 'female',
 30: 'radical',
 31: 'sexist',
 32: 'would',
 33: 'want',
 34: 'isi',
 35: 'even',
 36: 'go',
 37: 'time',
 38: 'thing',
 39: 'men',
 40: 'see',
 41: 'funny',
 42: 'never',
 43: 'good',
 44: 'need',
 45: 'bullying',
 46: 'hindu',
 47: 'also',
 48: 'quran',
 49: 'r',
 50: 'hate',
 51: 'world',
 52: 'girl',
 53: 'religion',
 54: 'man',
 55: 'country',
 56: 'really',
 57: 'still',
 58: 'jihad',
 59: 'guy',
 60: 'humanity',
 61: 'stop',
 62: 'said',
 63: 'love',
 64: 'ca',
 65: 'india',
 66: 'way',
 67: 'kat',
 68: 'making',
 69: 'many',
 70: 'made',
 71: 'take',
 72: 'got',
 73: 'look',
 74: 'mean',
 75: '

In [19]:
vocab_len = len(tok.index_word)
vocab_len

25037

In [20]:
seqtrain = tok.texts_to_sequences(xtrain) #step1
seqtrain

[[2, 691, 16, 322, 1198, 2639, 58, 10671, 240, 301, 711, 4935, 1442],
 [1199, 216, 11],
 [10672, 62, 639, 555, 4, 250, 712, 10673, 115, 47, 70, 3, 1, 2152],
 [2153,
  2,
  1276,
  241,
  25,
  34,
  14,
  30,
  20,
  111,
  657,
  155,
  5860,
  13,
  280,
  167,
  360,
  14,
  794,
  1443,
  13,
  280,
  112],
 [80, 2, 87, 5, 7, 62, 233, 99, 4268, 1386, 919, 90],
 [4269, 7415],
 [713, 10674, 5861, 4270, 15],
 [56,
  72,
  3398,
  80,
  2440,
  1166,
  466,
  4,
  54,
  35,
  495,
  3,
  1,
  57,
  28,
  99,
  640,
  62,
  1031,
  692,
  1444],
 [12, 10675, 819, 190, 1097, 199, 1098, 281, 304, 496, 8, 8],
 [12, 1922, 397, 1923, 1505, 1324, 3750, 18, 66, 152, 39, 3751],
 [125,
  161,
  156,
  1826,
  2285,
  42,
  234,
  2285,
  59,
  21,
  476,
  1571,
  59,
  238,
  205,
  84,
  5,
  22,
  69,
  2,
  4271,
  10,
  1445,
  7416,
  1200,
  292,
  1128,
  733],
 [10676, 136],
 [622, 501, 4272, 2286, 10677, 622, 146, 29, 29, 19, 27, 37],
 [82, 4936, 3752, 600, 7417, 153],
 [40, 2441, 440,

In [21]:
seqmattrain = sequence.pad_sequences(seqtrain, maxlen= int(max_len)) #step2
seqmattrain

array([[    0,     0,     0, ...,   711,  4935,  1442],
       [    0,     0,     0, ...,  1199,   216,    11],
       [    0,     0,     0, ...,     3,     1,  2152],
       ...,
       [    0,     0,     0, ...,  2530,    97,   534],
       [    0,     0,     0, ...,  1025, 25035, 25036],
       [    0,     0,     0, ...,    74,  1812,   411]], dtype=int32)

In [22]:
seqtest = tok.texts_to_sequences(xtest)
seqmattest = sequence.pad_sequences(seqtest, maxlen=int(max_len))

In [23]:
vocab_len

25037

In [24]:
rnn = Sequential()

rnn.add(Embedding(vocab_len+1,100, input_length=int(max_len), mask_zero=True))
rnn.add(SimpleRNN(units=32, activation="relu"))
rnn.add(Dense(units=32, activation="relu"))
rnn.add(Dropout(0.2))

rnn.add(Dense(units=1, activation="sigmoid"))

rnn.compile(optimizer="adam", loss="binary_crossentropy")

rnn.fit(seqmattrain, ytrain, batch_size=128, epochs=20)

ypred = rnn.predict(seqmattest)
ypred


Epoch 1/20
131/131 [==============================] - 31s 213ms/step - loss: 0.3917
Epoch 2/20
131/131 [==============================] - 14s 105ms/step - loss: 0.1688
Epoch 3/20
131/131 [==============================] - 12s 95ms/step - loss: 0.0818
Epoch 4/20
131/131 [==============================] - 8s 60ms/step - loss: 0.0420
Epoch 5/20
131/131 [==============================] - 10s 77ms/step - loss: 0.0266
Epoch 6/20
131/131 [==============================] - 9s 69ms/step - loss: 0.0200
Epoch 7/20
131/131 [==============================] - 9s 66ms/step - loss: 0.0175
Epoch 8/20
131/131 [==============================] - 8s 61ms/step - loss: 0.0146
Epoch 9/20
131/131 [==============================] - 8s 64ms/step - loss: 0.0091
Epoch 10/20
131/131 [==============================] - 7s 54ms/step - loss: 0.0090
Epoch 11/20
131/131 [==============================] - 8s 62ms/step - loss: 0.0080
Epoch 12/20
131/131 [==============================] - 8s 59ms/step - loss: 0.0084
Epoch 1

array([[9.1395378e-08],
       [2.6738828e-08],
       [9.9992037e-01],
       ...,
       [5.8858683e-19],
       [9.0209246e-15],
       [3.4276764e-03]], dtype=float32)

In [25]:
ypred = ypred>0.5
ypred = np.where(ypred<0.5,0,1)

In [26]:
from sklearn.metrics import classification_report
report = classification_report(ytest, ypred)
print(report)


              precision    recall  f1-score   support

           0       0.90      0.94      0.92      4758
           1       0.87      0.80      0.83      2417

    accuracy                           0.89      7175
   macro avg       0.89      0.87      0.88      7175
weighted avg       0.89      0.89      0.89      7175

